In [91]:
#!pip install fastparquet
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

## Import des jeux de données relatifs à l'affluence à bord (vélos et total)

In [9]:
# Chemins d'accès
data_path_C = r"C:\Users\9920510N\SNCF\Datalab' Mass Transit GrpO365 - Documents\Rayonnement\Datathon\2025\Hackathon_2025\Donnees\Course_et_comptage_Ligne_C_2022-2025.parquet"
data_path_U = r"C:\Users\9920510N\SNCF\Datalab' Mass Transit GrpO365 - Documents\Rayonnement\Datathon\2025\Hackathon_2025\Donnees\Course_et_comptage_Ligne_U_2022-2025.parquet"

data_path_velos = r"C:\Users\9920510N\SNCF\Datalab' Mass Transit GrpO365 - Documents\Rayonnement\Datathon\2025\Hackathon_2025\Donnees\Comptage vélos.csv"
data_path_emport = r"C:\Users\9920510N\SNCF\Datalab' Mass Transit GrpO365 - Documents\Rayonnement\Datathon\2025\Hackathon_2025\Donnees\Capacité emport vélos.csv"


In [161]:
# Ouverture des fichiers (et concaténation)
data_C = pd.read_parquet(data_path_C)
data_U = pd.read_parquet(data_path_U)
data_affluence = pd.concat([data_C,data_U])

data_velo = pd.read_csv(data_path_velos,sep=';')
data_velo['Code_gare'] = data_velo['Code_gare'].apply(lambda x: 547000 if x== 547026 else x) #erreur dans le referentiel de code gare sur Austerlitz
#data_emport = pd.read_csv(data_path_emport,sep=';')

In [162]:
display(data_affluence.head())

,Ligne,Année,Date,Jour,Numéro_marche,Gare,rang,Origine,Terminus,Mission,...,Montées,Descentes,Capacité_assise,Capacité_totale,Taux_occupation_assis,Taux_occupation_total,Composition_théorique,Nombre_voitures_théorique,Nombre_portes_théorique,Code_gare
0,C,2022,2022-11-19,SAM,140179,None,1.0,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,547000
1,C,2022,2022-11-20,DIM,140179,None,1.0,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,547000
2,C,2022,2022-12-11,DIM,140179,None,1.0,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,547000
3,C,2022,2022-10-02,DIM,140179,None,1.0,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,547000
4,C,2022,2022-04-02,SAM,140179,None,1.0,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,547000


In [163]:
display(data_velo.head())

,Code_gare,Sens_axe,Jour,Par_tranche_horaire,Ligne,Occupation_velo,Montees_velo,Descentes_velo
0,111278,Vers Nord,DIM,6,C,0.0,0.0,0.0
1,111278,Vers Nord,JOB,6,C,4.0,0.0,0.0
2,111278,Vers Nord,SAM,6,C,0.0,0.0,0.0
3,111278,Vers Nord,DIM,7,C,0.0,0.0,0.0
4,111278,Vers Nord,JOB,7,C,12.0,1.0,0.0


In [ ]:
display(data_emport.head())

## Pre-processing

#### Objectifs : Ajouter à la base affluence les informations de comptages et occupations vélo (passer d'une maille horaire à une maille train, en reartissant uniformément) et les capacités d'emport

In [164]:
#Ajout de la tranche horaire au départ sur la base affluence
data_affluence['Heure_arrivée_départ'] = data_affluence['Heure_départ_réalisé'].fillna(data_affluence['Heure_arrivée_réalisée']) #Pour les terminus, on a pas d'heure de départ
data_affluence['Par_tranche_horaire'] = data_affluence['Heure_arrivée_départ'].apply(lambda x: int(str(x)[7:9]))
display(data_affluence.head())

,Ligne,Année,Date,Jour,Numéro_marche,Gare,rang,Origine,Terminus,Mission,...,Capacité_assise,Capacité_totale,Taux_occupation_assis,Taux_occupation_total,Composition_théorique,Nombre_voitures_théorique,Nombre_portes_théorique,Code_gare,Heure_arrivée_départ,Par_tranche_horaire
0,C,2022,2022-11-19,SAM,140179,None,1.0,None,None,None,...,NaN,NaN,None,None,None,NaN,NaN,547000,0 days 22:13:00,22
1,C,2022,2022-11-20,DIM,140179,None,1.0,None,None,None,...,NaN,NaN,None,None,None,NaN,NaN,547000,0 days 22:19:28,22
2,C,2022,2022-12-11,DIM,140179,None,1.0,None,None,None,...,NaN,NaN,None,None,None,NaN,NaN,547000,0 days 22:17:42,22
3,C,2022,2022-10-02,DIM,140179,None,1.0,None,None,None,...,NaN,NaN,None,None,None,NaN,NaN,547000,0 days 22:13:00,22
4,C,2022,2022-04-02,SAM,140179,None,1.0,None,None,None,...,NaN,NaN,None,None,None,NaN,NaN,547000,0 days 22:15:36,22


In [165]:
#Ajout du sens en fonction de la parité (Attention la C et la U n'ont pas les même sens, mais bien la même parité)
data_affluence['Parité'] = data_affluence['Numéro_marche'].apply(lambda x: 'pair' if str(x)[-1] in ['0','2','4','6','8'] else 'impair')
data_affluence['Parité'] = data_affluence['Ligne'] +'_'+ data_affluence['Parité']
data_affluence['Sens_axe'] = data_affluence['Parité'].apply(lambda x: 'Vers Nord' if x=='C_pair' else ('Vers Sud' if x=='C_impair' else ('Vers Paris' if x=='U_pair' else 'U_impair')))


In [166]:
#Ajout du nombre de trains par heure à une gare de départ
data_nb_trains = data_affluence.groupby(['Code_gare','Par_tranche_horaire','Date','Sens_axe'],as_index=False).aggregate({"Numéro_marche":"count"}).rename(columns={"Numéro_marche":"Nb_Numéro_marche_heure"})
data_affluence_final = data_affluence.merge(data_nb_trains,on=['Code_gare','Par_tranche_horaire','Date','Sens_axe'],how='left')

data_affluence_final = data_affluence_final[data_affluence_final['Nb_Numéro_marche_heure']>0] #On supprime les heures sans trains
data_affluence_final = data_affluence_final.dropna(subset='Nb_Numéro_marche_heure') #On supprime les na
display(data_affluence_final.head())


,Ligne,Année,Date,Jour,Numéro_marche,Gare,rang,Origine,Terminus,Mission,...,Taux_occupation_total,Composition_théorique,Nombre_voitures_théorique,Nombre_portes_théorique,Code_gare,Heure_arrivée_départ,Par_tranche_horaire,Parité,Sens_axe,Nb_Numéro_marche_heure
0,C,2022,2022-11-19,SAM,140179,None,1.0,None,None,None,...,None,None,NaN,NaN,547000,0 days 22:13:00,22,C_impair,Vers Sud,10
1,C,2022,2022-11-20,DIM,140179,None,1.0,None,None,None,...,None,None,NaN,NaN,547000,0 days 22:19:28,22,C_impair,Vers Sud,8
2,C,2022,2022-12-11,DIM,140179,None,1.0,None,None,None,...,None,None,NaN,NaN,547000,0 days 22:17:42,22,C_impair,Vers Sud,8
3,C,2022,2022-10-02,DIM,140179,None,1.0,None,None,None,...,None,None,NaN,NaN,547000,0 days 22:13:00,22,C_impair,Vers Sud,8
4,C,2022,2022-04-02,SAM,140179,None,1.0,None,None,None,...,None,None,NaN,NaN,547000,0 days 22:15:36,22,C_impair,Vers Sud,8


In [167]:
#Ajout des comptages vélos par tranche horaire dans la base affluence
data_affluence_merge = data_affluence_final.merge(data_velo,on=['Code_gare','Par_tranche_horaire','Jour','Sens_axe','Ligne'],how='left')
display(data_affluence_merge.head())


,Ligne,Année,Date,Jour,Numéro_marche,Gare,rang,Origine,Terminus,Mission,...,Nombre_portes_théorique,Code_gare,Heure_arrivée_départ,Par_tranche_horaire,Parité,Sens_axe,Nb_Numéro_marche_heure,Occupation_velo,Montees_velo,Descentes_velo
0,C,2022,2022-11-19,SAM,140179,None,1.0,None,None,None,...,NaN,547000,0 days 22:13:00,22,C_impair,Vers Sud,10,14.0,7.0,0.0
1,C,2022,2022-11-20,DIM,140179,None,1.0,None,None,None,...,NaN,547000,0 days 22:19:28,22,C_impair,Vers Sud,8,11.0,4.0,1.0
2,C,2022,2022-12-11,DIM,140179,None,1.0,None,None,None,...,NaN,547000,0 days 22:17:42,22,C_impair,Vers Sud,8,11.0,4.0,1.0
3,C,2022,2022-10-02,DIM,140179,None,1.0,None,None,None,...,NaN,547000,0 days 22:13:00,22,C_impair,Vers Sud,8,11.0,4.0,1.0
4,C,2022,2022-04-02,SAM,140179,None,1.0,None,None,None,...,NaN,547000,0 days 22:15:36,22,C_impair,Vers Sud,8,14.0,7.0,0.0


In [168]:
#Diviser les chiffres relatifs aux vélos par nombre de trains
data_affluence_merge['Occupation_velo'] = data_affluence_merge['Occupation_velo']/data_affluence_merge['Nb_Numéro_marche_heure']
data_affluence_merge['Montees_velo'] = data_affluence_merge['Montees_velo']/data_affluence_merge['Nb_Numéro_marche_heure']
data_affluence_merge['Descentes_velo'] = data_affluence_merge['Descentes_velo']/data_affluence_merge['Nb_Numéro_marche_heure']
data_affluence_merge = data_affluence_merge.dropna(subset=['Occupation_velo','Montees_velo','Descentes_velo']) #On supprime les na

data_affluence_merge['Occupation_velo'] = data_affluence_merge['Occupation_velo'].astype(int)
data_affluence_merge['Montees_velo'] = data_affluence_merge['Montees_velo'].astype(int)
data_affluence_merge['Descentes_velo'] = data_affluence_merge['Descentes_velo'].astype(int)

In [169]:
display(data_affluence_merge['Montees_velo'].sum())

np.int64(268229)

In [170]:
#Ajout des données de capacités vélos
#On suppose qu'on peut rentrer un vélo par porte et des vélos supplémentaires si y a des places dédiées (pour la C par exemple)
data_affluence_merge['Capacité_totale_velo'] = data_affluence_merge['Ligne'].apply(lambda x: 3 if x=='N' else 0) #4 caisses US CU en majorité, avec 2 portes par caisses, 6 caisses pour la N avec 1 porte par caisse
data_affluence_merge['Nb_caisses'] = data_affluence_merge['Nombre_portes_théorique'].apply(lambda x: 1 if x<10 else 2) #On précise ici combien il y a de rames

#Pour ensuite venir multiplier les places dédiées par le nombre de rames en y ajoutant le nb de places aka le nb de portes
data_affluence_merge['Capacité_totale_velo'] = data_affluence_merge['Capacité_totale_velo']*data_affluence_merge['Nb_caisses'] + data_affluence_merge['Nombre_portes_théorique']

data_affluence_merge['Reste_capacite_velo'] = data_affluence_merge['Capacité_totale_velo'] - data_affluence_merge['Occupation_velo']
data_affluence_merge['Reste_capacite_velo'] = data_affluence_merge['Reste_capacite_velo'].apply(lambda x: 0 if x<0 else x) #On évite les cas négatifs
data_affluence_merge['Taux_occupation_velo'] = 1-(data_affluence_merge['Reste_capacite_velo']/data_affluence_merge['Capacité_totale_velo'])

    

In [171]:
#Filtrer le dataframe sur les colonnes utiles
data_affluence_clean = data_affluence_merge[['Ligne', 'Année', 'Date', 'Jour', 'Numéro_marche', 'Gare', 'Charge', 'Montées',
       'Descentes', 'Capacité_assise', 'Capacité_totale',
       'Taux_occupation_assis', 'Taux_occupation_total', 'Code_gare', 'Heure_arrivée_départ',
       'Par_tranche_horaire', 'Parité', 'Sens_axe', 'rang',
       'Occupation_velo', 'Montees_velo', 'Descentes_velo','Capacité_totale_velo','Reste_capacite_velo','Taux_occupation_velo']]

## Calcul de l'indicateur (voir matrice du calcul en annexe)

In [172]:
#Construction d'une matrice numérique, chaque chiffre correspond à une situation d'affluence combinée vélo+personnes pour laquelle 
#on associe un niveau de couleur

matrice = pd.DataFrame({'Valeur':[101,102,103,201,202,203,301,302,303],'Score':[3,3,2,3,2,1,2,1,1]})
dict_matrice = dict(zip(matrice['Valeur'],matrice['Score']))

In [173]:
#Ajout des valeurs numériques de la matrice, pour les velos, pour les personnes et enfin la somme des 2
data_affluence_clean['Valeur affluence velo'] = data_affluence_clean['Taux_occupation_velo'].apply(lambda x: 100 if x>0.5 else (300 if x<0.25 else 200))

data_affluence_clean['Taux_occupation_total'] = data_affluence_clean['Taux_occupation_total'].fillna('0.0').apply(lambda x: float(x.replace(',','.')))
data_affluence_clean['Valeur affluence'] = data_affluence_clean['Taux_occupation_total'].apply(lambda x: 1 if x>0.5 else (3 if x<0.25 else 2))
data_affluence_clean['Valeur'] = data_affluence_clean['Valeur affluence velo'] + data_affluence_clean['Valeur affluence']


In [174]:
#Ajout du score en fonction de la matrice de correspondance
data_affluence_clean['Score'] = data_affluence_clean['Valeur'].map(dict_matrice)
display(data_affluence_clean.head())

,Ligne,Année,Date,Jour,Numéro_marche,Gare,Charge,Montées,Descentes,Capacité_assise,...,Occupation_velo,Montees_velo,Descentes_velo,Capacité_totale_velo,Reste_capacite_velo,Taux_occupation_velo,Valeur affluence velo,Valeur affluence,Valeur,Score
0,C,2022,2022-11-19,SAM,140179,None,NaN,NaN,NaN,NaN,...,1,0,0,NaN,NaN,NaN,200,3,203,1
1,C,2022,2022-11-20,DIM,140179,None,NaN,NaN,NaN,NaN,...,1,0,0,NaN,NaN,NaN,200,3,203,1
2,C,2022,2022-12-11,DIM,140179,None,NaN,NaN,NaN,NaN,...,1,0,0,NaN,NaN,NaN,200,3,203,1
3,C,2022,2022-10-02,DIM,140179,None,NaN,NaN,NaN,NaN,...,1,0,0,NaN,NaN,NaN,200,3,203,1
4,C,2022,2022-04-02,SAM,140179,None,NaN,NaN,NaN,NaN,...,1,0,0,NaN,NaN,NaN,200,3,203,1


In [175]:
display(data_affluence_clean['Score'].value_counts()) #Répartition par score

Score
1    13233650
2      139437
3        3348
Name: count, dtype: int64

In [176]:
display(data_affluence_clean['Valeur'].value_counts()) #Répartion par valeur

Valeur
303    10049381
203     2311088
302      873181
301      110416
202       23181
103        5840
201        2293
102         993
101          62
Name: count, dtype: int64

## Export de la base


In [107]:
save_path = r"C:\Users\9920510N\SNCF\Datalab' Mass Transit GrpO365 - Documents\Rayonnement\Datathon\2025\Hackathon_2025\Donnees\Scoring_Affluence_velo.csv"
data_affluence_clean.to_csv(save_path,sep=';')


In [177]:
#Filtrage sur 2025 pour tableau de bord
data_affluence_clean_2025 = data_affluence_clean[data_affluence_clean['Année']==2025]
display(data_affluence_clean_2025.shape)

(2104311, 29)

In [178]:
data_affluence_clean_2025['Mois'] = data_affluence_clean_2025['Date'].apply(lambda x: int(str(x).split('-')[1]))
data_affluence_clean_2025_06 = data_affluence_clean_2025[data_affluence_clean_2025['Mois']==6]

In [179]:
save_path_juin = r"C:\Users\9920510N\SNCF\Datalab' Mass Transit GrpO365 - Documents\Rayonnement\Datathon\2025\Hackathon_2025\Donnees\Scoring_Affluence_velo_juin.csv"
data_affluence_clean_2025_06.to_csv(save_path_juin,sep=';')

In [180]:
save_path_bi = r"C:\Users\9920510N\SNCF\Datalab' Mass Transit GrpO365 - Documents\Rayonnement\Datathon\2025\Hackathon_2025\Donnees\Scoring_Affluence_velo_BI.csv"
data_affluence_clean_2025.to_csv(save_path_bi,sep=';')


In [181]:
data_test = pd.read_csv(save_path_bi,sep=';')